<b>FLUX TEXT-To-Image/ Img-to-Img deployment</b>

In [8]:
# conda create -n torch_env python=3.10 -y
# conda activate torch_env
# pip install torchvision 
# pip install sentencepiece protobuf
# After doing this outside in a terminal restart the kernel for this notebook

Channels:
 - https://aws-ml-conda.s3.us-west-2.amazonaws.com
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda/envs/torch_env

  added / updated specs:
    - python=3.10


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h4bc722e_7 
  ca-certificates    conda-forge/linux-64::ca-certificates-2024.7.4-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-hf3520f5_7 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng-14.1.0-h77fa898_0

In [5]:
# Fire this so Jupyter opens with Conda-Torch-Env kernel
# source activate myenv
# pip install ipykernel
# python -m ipykernel install --user --name torch_env --display-name "Conda-torch_env"
# After you've execute this select Conda-torch_env kernel for this notebook

SyntaxError: invalid syntax (1909789493.py, line 1)

In [3]:
pip install git+https://github.com/huggingface/diffusers.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-noxo9l9t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-noxo9l9t
  Resolved https://github.com/huggingface/diffusers.git to commit 65e30907b5df1338ae65a20b78866c87e061c952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install transformers accelerate
# After this i had to restart the kernel for the modules to reflect

In [2]:
import torch
from diffusers import FluxPipeline

In [4]:
model_id = "black-forest-labs/FLUX.1-schnell" #you can also use `black-forest-labs/FLUX.1-dev`

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
#save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

Loading pipeline components...: 100%|█████████████| 7/7 [00:01<00:00,  4.42it/s]


In [5]:
pipe

FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.30.0.dev0",
  "_name_or_path": "black-forest-labs/FLUX.1-schnell",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
prompt = "A cat holding a sign that says hello world"
seed = 42
image = pipe(
    prompt,
    output_type="pil",
    num_inference_steps=4, #use a larger number if you are using [dev]
    generator=torch.Generator("cuda").manual_seed(seed)
).images[0]
image.save("flux-schnell.png")

In [10]:
!pip install optimum-quanto

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.2 MB/s eta 0:00:00


In [1]:
# import torch
# from diffusers import FluxTransformer2DModel, FluxPipeline
# from transformers import T5EncoderModel, CLIPTextModel
# from optimum.quanto import freeze, qfloat8, quantize

# bfl_repo = "black-forest-labs/FLUX.1-dev"
# dtype = torch.bfloat16

# transformer = FluxTransformer2DModel.from_single_file("https://huggingface.co/Kijai/flux-fp8/blob/main/flux1-dev-fp8.safetensors", torch_dtype=dtype)
# quantize(transformer, weights=qfloat8)
# freeze(transformer)

# text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype)
# quantize(text_encoder_2, weights=qfloat8)
# freeze(text_encoder_2)

# pipe = FluxPipeline.from_pretrained(bfl_repo, transformer=None, text_encoder_2=None, torch_dtype=dtype)
# pipe.transformer = transformer
# pipe.text_encoder_2 = text_encoder_2

# pipe.enable_model_cpu_offload()

# prompt = "A cat holding a sign that says hello world"
# image = pipe(
#     prompt,
#     guidance_scale=3.5,
#     output_type="pil",
#     num_inference_steps=20,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]

# image.save("flux-fp8-dev.png")

/opt/conda/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: black-forest-labs/FLUX.1-dev is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `token` or log in with `huggingface-cli login`.

In [ ]:
from IPython.display import Image, display

In [ ]:
display("flux-schnell.png")